<a href="https://colab.research.google.com/github/delibae/health_care_pr/blob/minsoo/Data_Load%2BResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import glob
import pickle
import json
import os
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
input_path = ['/content/drive/MyDrive/data/두피/Image_hairloss/*.jpg', 
              '/content/drive/MyDrive/data/두피/Image_keratosis/*.jpg',
              '/content/drive/MyDrive/data/두피/Image_dendruff/*.jpg',
            ]

In [ ]:
def make_path(path) :

    result = []

    for i in path :
        
        for j in glob.glob(i) :
            result.append(j)

    return result

In [ ]:
image = make_path(input_path)

len(image)

12564

In [ ]:
def handle_image(path) :

    reshaped_image = []

    for i in tqdm(path) :
        
        image_bgr = cv2.imread(i, cv2.IMREAD_COLOR)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        image_50x50 = cv2.resize(image_rgb, (50, 50))

        reshaped_image.append(image_50x50)

    return reshaped_image

image = handle_image(image)
image = np.array(image)

100%|██████████| 12564/12564 [38:09<00:00,  5.49it/s]


In [ ]:
label_path = ['/content/drive/MyDrive/data/두피/Label_hairloss/*.json',
              '/content/drive/MyDrive/data/두피/Label_keratosis/*.json',
              '/content/drive/MyDrive/data/두피/Label_dendruff/*.json', 
            ]

label = make_path(label_path)
len(label)

12564

In [ ]:
def handle_json(path) :

    result = []

    for p in tqdm(path) :
        with open(p, 'r',encoding="UTF-8") as file:
            data = json.load(file)
            data_path = data['image_file_name']
            value = []
            for i in range(1, 7) :
                k = f'value_{i}'
                value.append(int(data[k]))
          
            result.append(value)

    return result
        
label = handle_json(label)
label = np.array(label)

  0%|          | 0/12564 [00:00<?, ?it/s]


TypeError: ignored

In [ ]:
def softmax(data) :
    exp_data = np.exp(data)

    return exp_data / np.sum(exp_data, axis=1, keepdims=True)

softmax_data = softmax(label)

assert np.isclose(np.sum(softmax_data, axis=1), np.ones(label.shape[0])).all()

label = softmax_data

In [ ]:
image = image.reshape(-1, 3, 50, 50)

In [ ]:
image.shape

torch.Size([12564, 3, 50, 50])

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

image = torch.tensor(image)
label = torch.tensor(label)

<ipython-input-99-feb1b66f1136>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image)
<ipython-input-99-feb1b66f1136>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image, label, test_size=0.33, random_state=42)

In [ ]:
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
import torchvision

model = torchvision.models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_dataloader, 0)) :
        inputs, labels = data

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    total = 0
    correct = 0
    accuracy = []
    test_loss = 0.0

    for j, data in enumerate(tqdm(test_dataloader, 0) :
        inputs, labels = data

        # 결과값 연산
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        test_loss += loss.item()

    print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f' %(epoch+1, num_epochs, running_loss / (i + 1), test_loss / (j + 1)))

print('Finished Training')

  3%|▎         | 7/264 [00:06<04:10,  1.02it/s]


KeyboardInterrupt: ignored